In [ ]:
import os,sys
import mne
import json
import matplotlib.pyplot as plt
import numpy as np

code_dir = os.path.expandvars('$OSCBAGDIS_DATAPROC_CODE')
sys.path.append(code_dir)

import utils  #my code
import utils_preproc as upre
from os.path import join as pjoin
%load_ext autoreload
%autoreload 2



if os.environ.get('DATA_DUSS') is not None:
    data_dir = os.path.expandvars('$DATA_DUSS')
else:
    data_dir = '/home/demitau/data'
    

rawname_ = 'S01_off_hold'
#rawname_ = 'S01_on_hold'
#rawname_ = 'S01_off_move'
#rawname_ = 'S01_on_move'

#rawname_ = 'S02_off_hold'
#rawname_ = 'S02_on_hold' 
#rawname_ = 'S02_off_move'  
#rawname_ = 'S02_on_move'

#rawname_ = 'S03_off_move'  
#rawname_ = 'S03_off_hold'  

rawname = rawname_ + '_resample_raw.fif'
fname_full = pjoin(data_dir,rawname)

subj,medcond,task  = utils.getParamsFromRawname(rawname_)
sind_str = subj


# read file -- resampled to 256 Hz,  Electa MEG, EMG, LFP, EOG channels
raw = mne.io.read_raw_fif(fname_full, None)
mod_info, infos = upre.readInfo(rawname_, raw)
raw.info = mod_info

# get info about bad MEG channels (from separate file)
with open(pjoin(code_dir,'subj_info.json') ) as info_json:
        #raise TypeError

    #json.dumps({'value': numpy.int64(42)}, default=convert)
    gen_subj_info = json.load(info_json)
    
maintremside = gen_subj_info[subj]['tremor_side']
nonmaintremside = utils.getOppositeSideStr(maintremside)
    
badchlist = gen_subj_info[subj]['bad_channels'][medcond][task]
raw.info['bads'] = badchlist
print('bad channels (from Jan) are ',badchlist)

In [ ]:
fname_bads = '{}_MEGch_bads_upd.npz'.format(rawname_)
fname_bads_full = os.path.join( data_dir, fname_bads)
if os.path.exists(fname_bads_full):
    badchlist_upd = list(np.load(fname_bads_full)['arr_0'])
    print('Setting additional bad channels ', set(badchlist_upd)-set(badchlist))

    raw.info['bads'] = list(badchlist_upd)
    
    do_MaxFilter_bad_chans = 0
else:
    do_MaxFilter_bad_chans = 1
print(do_MaxFilter_bad_chans)

In [ ]:
# Butterwordth 4th order -- for stopping power grid noise
#help(raw_sss.notch_filter)
raw.load_data()
#import numpy as np
freqsToKill = np.arange(50, 128, 50) # harmonics of 50
raw.notch_filter(freqsToKill, picks=['meg','eog'])

In [ ]:
radius, origin, _ = mne.bem.\
fit_sphere_to_headshape(mod_info, dig_kinds=('cardinal','hpi'))

In [ ]:
fine_cal_file  = os.path.join(data_dir,  'sss_cal.dat')
crosstalk_file = os.path.join(data_dir,  'ct_sparse.fif')

In [ ]:
#Maxwell filter
raw_sss = mne.preprocessing.\
maxwell_filter(raw, cross_talk=crosstalk_file,
               calibration=fine_cal_file, origin=origin,
              coord_frame='head')

import gc; gc.collect()

In [ ]:
raw.info['bads']

In [ ]:
raw_ssp = raw.copy()

In [ ]:
raw_ssp.info['projs']

In [ ]:
raw_ssp.apply_proj()
raw_ssp.info['bads'] = []

In [ ]:
raw_ssp.info

In [ ]:
#Maxwell filter
raw_tsss = mne.preprocessing.\
maxwell_filter(raw, cross_talk=crosstalk_file,
               calibration=fine_cal_file, origin=origin,
              coord_frame='head', st_duration=10)

import gc; gc.collect()

In [ ]:
#Maxwell filter
raw_sss_frame_meg = mne.preprocessing.\
maxwell_filter(raw, cross_talk=crosstalk_file,
               calibration=fine_cal_file, coord_frame='meg')

import gc; gc.collect()

In [ ]:
raw.info

In [ ]:
raw_tsss.info['projs']

In [ ]:
raw.info['projs']

In [ ]:
help(raw_sss.apply_proj)

In [ ]:
raw_sss.info

In [ ]:
raws_toshow = {'naked':raw.copy(), 'sss':raw_sss, 'sss_fm':raw_sss_frame_meg,
               'tsss':raw_tsss, 'ssp':raw_ssp}

In [ ]:
raws_toshow['naked'].info['bads'] = []

In [ ]:
# raw_sss_frame_meg.pick_types(meg=True, misc=False, bio=False, emg=False, 
#                               eog=False, exclude='bads')
# raw_sss_frame_meg = raw_sss_frame_meg.filter(l_freq=1., h_freq=None)

In [ ]:
#raws_toshow['sss_fm'] = raw_sss_frame_meg

In [ ]:
for k in raws_toshow:
    raws_toshow[k].load_data()
    raws_toshow[k].pick_types(meg=True, misc=False, bio=False, emg=False, 
                              eog=False, exclude='bads')
    raws_toshow[k] = raws_toshow[k].filter(l_freq=1., h_freq=None)
    # get rid of bad channels

In [ ]:
rectconv_fname_full = os.path.join(data_dir, '{}_emg_rectconv.fif'.format(rawname_) )
raw_emg_rectconv = mne.io.read_raw_fif(rectconv_fname_full)


# Plot overall

In [ ]:
%matplotlib qt
mult = 2.5
import utils_tSNE as utsne



In [ ]:
len(raws_toshow['naked'].ch_names)

In [ ]:
badchlist_upd

In [ ]:
nonbad = list( set(raws_toshow['naked'].ch_names) - set(badchlist_upd) )
len(nonbad)

In [ ]:
plt.close('all')
ww = 14; hh = 4
nr = len(raws_toshow)+1; nc = 1
fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr*hh),sharex='col')
ks = list(raws_toshow.keys())


xlim = raw.times[0],raw.times[-1]
xlim = 300,400
xlim = 500,550

scale_mode = 'separate'
scale_mode = 'from_naked'
#scale_mode = 'no'

chansel = nonbad
#chansel = raws_toshow['naked'].ch_names

ax = axs[0]
emg = raw_emg_rectconv.get_data()
for emgchi in range(len(emg)):
    ax.plot(raw.times,emg[emgchi], label=raw_emg_rectconv.ch_names[emgchi], alpha=0.6)
ax.legend(loc='upper right')


megdat,times = raws_toshow['naked'][nonbad]
me_naked, mn_naked,mx_naked = utsne.robustMean(megdat, axis=1, per_dim =1, 
                                               ret_aux=1, q = .25)
# megdat_sum = np.sum(np.abs(megdat_scaled),axis=0)
# me_s_naked, mn_s_naked, mx_s_naked = utsne.robustMean(megdat_sum, axis=None, 
#                                    per_dim =1, ret_aux=1, pos = 1)

for i,k in enumerate(ks):
    #raw_only_meg = filt_raw.copy()
    #megdat = raws_toshow[k].get_data()
    megdat,times = raws_toshow[k][nonbad]
    
    # pctshift = 25
    # pcts = [pctshift, 100-pctshift]
    # qs = np.percentile(megdat, pcts, axis=1)
    me, mn,mx = utsne.robustMean(megdat, axis=1, per_dim =1, ret_aux=1, q = .25)
    if scale_mode == 'separate':
        megdat_scaled = ( megdat - me[:,None] ) 
        megdat_scaled /= (mx-mn)[:,None]
    elif scale_mode == 'from_naked':
        megdat_scaled = ( megdat - me_naked[:,None] ) 
        megdat_scaled /= (mx_naked - mn_naked)[:,None]
    elif scale_mode == 'no':
        megdat_scaled = ( megdat - me[:,None] ) 
    megdat_sum = np.sum(np.abs(megdat_scaled),axis=0)
    me_s, mn_s,mx_s = utsne.robustMean(megdat_sum, axis=None, 
                                       per_dim =1, ret_aux=1, pos = 1)

    mask= megdat_sum > mx_s * mult
    _,ivals_meg_artif = utils.getIntervals(np.where(mask)[0] ,include_short_spikes=1, 
                                           endbin=len(mask), thr=1e-2)
    ivals_meg_artif
    
    
    ax = axs[i+1]
    ax.plot(raw.times,megdat_sum)
    ax.axhline( me_s , c='r', ls=':')
    ax.axhline( mx_s , c='purple', ls=':')
    ax.axhline( me_s * mult , c='r', ls=':')
    ax.axhline( mx_s * mult , c='purple', ls=':')
    ax.set_title(k)

    for ivl in ivals_meg_artif:
        b0,b1 = ivl
        ax.axvline( raw.times[b0] , c='r', ls=':')
        ax.axvline( raw.times[b1] , c='r', ls=':')
        
    ax.set_xlim(xlim)
        
plt.suptitle('scale = {}'.format(scale_mode))
plt.savefig('{}_multiArtifCorr_dur{:.1f}s.png'.format(rawname_,xlim[1]-xlim[0] ),dpi=200)

# Plot power concentr

In [ ]:
sfreq = raw.info['sfreq']
#decim = 10
decim = 30
freq_min, freq_max = 3,90
freq_step = 1
freqs = np.arange(freq_min,freq_max,freq_step) #np.linspace(freq_min, freq_max, num=nfreqs)
endrec = raw.times[-1]

n_cycles = 7 + freqs  #/ 2.  # different number of cycle per frequency

In [ ]:
tfrs = {}

In [ ]:
raws_toshow.keys()

In [ ]:
for k in ks:
    strec = 0
    #endrec = 300  
    epdur = endrec
    curraw=  raws_toshow[k]
    new_events = mne.make_fixed_length_events(curraw,\
        start=strec, stop=endrec, duration=epdur)
    epochs = mne.Epochs(curraw,new_events, tmin=0,
                        tmax = epdur, baseline=None)

    # sh=2
    # cfq = 15
    # freqs = np.logspace(*np.log10([cfq-sh,cfq+sh ]), num=6)
    power = mne.time_frequency.tfr_morlet(epochs,\
        freqs=freqs, n_cycles=n_cycles, use_fft=True,\
        return_itc=False, decim=decim, n_jobs=10)
    tfrs[k] = power

In [ ]:
plt.close('all')

In [ ]:
data_band.shape

In [ ]:
data_band_me.shape

In [ ]:
data_band.shape

In [ ]:
#maintremside

In [ ]:
fbs = [ ('tremfreq', 3,9), ('alpha/mu', 8,13),  ('beta', 15,30),  ('gamma', 30,90)  ]

In [ ]:
#select some interval types and restrict by time
reg_intervals = 1
if reg_intervals:
    intlen_s = 30
    timeints = [(0,endrec,'entire')]
    int_bounds = np.arange(raw.times[0],raw.times[-1],intlen_s)
    for ib in range(1,len(int_bounds)):
        st,end = int_bounds[ib-1],int_bounds[ib]
        it = 'reg{}_{}'.format(intlen_s,ib)
        timeints += [(st,end,it)]
else:
    timeints_ = [(0,endrec,'entire')]

    for t in ['trem', 'hold', 'move', 'notrem']:
        for side in ['L', 'R']:
            it = '{}_{}'.format(t,side)
            print(it, ivalis.get(it,[]))
            timeints_ += ivalis.get(it,[])

    timeints = []
    for i,ti in enumerate(timeints_):
        timin,timax,tiname = ti
        if timin <= endrec - decim /sfreq:
            timeints += [ti]
        
print(len(timeints))

In [ ]:
anns_fn = rawname_ + '_anns.txt'
anns_fn_full = os.path.join(data_dir, anns_fn)
anns = mne.read_annotations(anns_fn_full)

In [ ]:
for i,k in enumerate(ks):
    utils.plotBandLocations(tfrs[k],\
    timeints,fbs,prefix=rawname_+'_corr_{}_regint{}_'.format(k,reg_intervals),\
    anns=anns)
    plt.close()

In [ ]:
gc.collect()

In [ ]:
ww = 14; hh = 4
nr = len(raws_toshow) + 1; nc = 1
fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr*hh))

ax = axs[0]
emg = raw_emg_rectconv.get_data()
for emgchi in range(len(emg)):
    ax.plot(raw.times,emg[emgchi], label=raw_emg_rectconv.ch_names[emgchi], alpha=0.6)
ax.legend(loc='upper right')

for i,k in enumerate(ks):
#     if i > 0: 
#         break
    
    tfr=  tfrs[k]
    ax =axs[i+1]
    
    logscale = 1
    
    fbmin,fbmax = 0,90
    #interval = ( raw.times[0], raw.times[-1], 'entire')
    interval = ( 350,460, 'bad')
    fb = ('allf', fbmin, fbmax)
    
    freqi = np.where( (tfr.freqs <= fbmax) * (tfr.freqs >=fbmin) )[0]
    data_band = np.abs(tfr.data)[:,freqi[0]:freqi[-1]+1,:]
    if logscale:
        data_band = np.log(data_band)
        data_band -= np.min(data_band,axis=2)[:,:,None]
    #data_band_me = np.mean( np.mean(data_band, axis=2), axis=1 )  #mean over freq
    data_band_me = np.mean( np.mean(data_band, axis=2), axis=1 )  #mean over freq
    #data_band_me = np.mean(np.mean(data, axis=2), axis=1)

    qoffset=  5e-2
    #qoffset=  1e-2
    mn = np.quantile(data_band_me, qoffset)
    mx = np.quantile(data_band_me, 1 - qoffset)
    print(k, mn,mx, len(freqi))

    utils.plotTopomapTau(ax, tfr, interval, fb, mn,mx, logscale=logscale, colorbar=1)
    ax.set_title(k)

    
    #tfr.plot_topo(dB=False, mode='zlogratio', baseline=(None,None), vmin=-2,vmax=2)

In [ ]:
raw.times[-1]

In [ ]:
mn,mx

In [ ]:
gc.collect()
